# 機器學習作業

1. 資料來源：[https://www.kaggle.com/competitions/boston20200827/](https://www.kaggle.com/competitions/boston20200827/)
2. 要求：
- 完整的機器學習預測流程,包括:資料前處理、視覺化、模型訓練、預測、結果評估等。
- 至少使用兩種模型並比較其效果。

## 資料欄位說明

| 欄位      | 說明                                           |
|-----------|------------------------------------------------|
| ID        | 資料的唯一識別碼。                             |
| CRIM      | 每人均犯罪率。                                 |
| ZN        | 佔地面積超過 25,000 平方英尺的住宅用地比例。   |
| INDUS     | 每鎮非零售業務用地的比例。                     |
| CHAS      | 查爾斯河虛擬變數（= 1 如果地段鄰近河流；否則為 0）。|
| NOX       | 一氧化氮濃度（每 10 百萬分之一）。             |
| RM        | 每棟住宅的平均房間數。                         |
| AGE       | 1940 年之前建造的自住單位比例。                 |
| DIS?      | 到波士頓五個就業中心的加權平均距離。           |
| RAD       | 到放射狀公路的可達性指數。                     |
| TAX       | 每 10,000 美元的房產稅率。                     |
| PTRATIO   | 每個城鎮的學生與教師比例。                     |
| B 1000    | 1000(Bk - 0.63)^2，其中 Bk 是鎮上黑人比例。    |
| LSTAT     | 低收入人口的比例（百分比）。                   |
| PRICE     | 自住單位的中位數價格（以千美元計）。           |


推測自：

[https://www.kaggle.com/competitions/boston-dataset/data?select=boston_data.csv](https://www.kaggle.com/competitions/boston-dataset/data?select=boston_data.csv)

[https://www.kaggle.com/datasets/altavish/boston-housing-dataset/data](https://www.kaggle.com/datasets/altavish/boston-housing-dataset/data)


### 我的發現

搞了一堆有的沒的都是負優化，最後決定只對資料做 StandardScaler，這是我能找出來的最好的結果。

嘗試過：

- 共線性特徵移除其一
- 移除 'ID', 'CHAS' 欄位，邏輯上不應該影響房價
- SelectKBest: k=2~13 都試過
- z-score 移除離群值

## 載入資料

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
Boston_train_df = pd.read_csv(r'./boston20200827/Boston_train.csv')
Boston_test_df = pd.read_csv(r'./boston20200827/Boston_test.csv')
df = pd.concat([Boston_train_df, Boston_test_df], sort=False)

df.head()

In [ ]:
df.describe()

In [ ]:
print(df.info())

## 資料視覺化

In [ ]:
# Let's see how data is distributed for every column

plt.figure(figsize = (20, 15))
plotnumber = 1

for column in df:
    if plotnumber <= 14:
        ax = plt.subplot(3, 5, plotnumber)
        sns.histplot(df[column])
        plt.xlabel(column, fontsize = 15)
        
    plotnumber += 1
    
plt.tight_layout()
plt.show()

In [ ]:
# Plotting `Price` with remaining columns

plt.figure(figsize = (20, 15))
plotnumber = 1

for column in df:
    if plotnumber <= 14:
        ax = plt.subplot(3, 5, plotnumber)
        sns.scatterplot(x = df['PRICE'], y = df[column])
        
    plotnumber += 1

plt.tight_layout()
plt.show()

In [ ]:
sns.displot(df['PRICE'])
plt.show()

In [ ]:
# looking for outliers using box plot

plt.figure(figsize = (20, 8))
sns.boxplot(data = df, width = 0.8)
plt.show()

## 資料前處理

In [ ]:
df.isnull().sum()

模型的目標是預測房價（PRICE），因此缺少目標值的數據對於訓練模型來說是沒有意義的。

這些缺失目標值的樣本無法提供有用的信息來幫助模型學習，因此應該將它們從訓練數據中移除。

In [ ]:
df = df.dropna(subset=['PRICE'])
df.isnull().sum()

In [ ]:
X = df.drop(['PRICE'], axis=1)
y = df['PRICE']

### 異常值處理

There are some outliers in data.

標準化（Standardization）可以在一定程度上減少離群值對數據的影響，因為它將數據縮放到一個相對一致的範圍內。

In [ ]:
# from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

In [ ]:
plt.figure(figsize = (20, 8))
sns.boxplot(data = X_scaled, width = 0.8)
plt.show()

#### Z-Score 移除異常值

In [ ]:
# import numpy as np

# # 計算 Z-score
# z_scores = np.abs((X_scaled - X_scaled.mean()) / X_scaled.std())

# # 設定 Z-score 閾值，通常為 3
# threshold = 3
# outliers = np.where(z_scores > threshold)

# # 移除異常值
# X_clean = np.delete(X_scaled, outliers[0], axis=0)
# y_clean = np.delete(y, outliers[0], axis=0)

# plt.figure(figsize = (20, 8))
# sns.boxplot(data = X_clean, width = 0.8)
# plt.show()

# X_scaled = X_clean
# y = y_clean

### 檢查多重共線性

- 不穩定的係數估計：當自變數之間高度相關時，回歸係數的估計值會變得不穩定，可能會出現很大的標準誤差。
- 解釋困難：高度相關的自變數使得很難確定哪個變數對應變數有實質性的影響。


In [ ]:
# checking for multicollinearity using `VIF` and `correlation matrix`

# from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()

vif['VIF'] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]
vif['Features'] = X.columns

vif

In [ ]:
# Heatmap

fig, ax = plt.subplots(figsize = (16, 8))
sns.heatmap(df.corr(), annot = True, fmt = '1.2f', annot_kws = {'size' : 10}, linewidth = 1)
plt.show()

RAD 和 TAX 之間的相關性很高，因此我們可以考慮移除其中一個變數。

In [ ]:
# dropping 'RAD' column from data

# X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# X_scaled_df = X_scaled_df.drop(columns=['RAD'], axis=1)

#### SelectKBest

用途：選擇K個最好的特徵，並且可以通過設置不同的參數來選擇不同的特徵選擇方法。

In [ ]:
# from sklearn.feature_selection import SelectKBest

# selectkbest = SelectKBest(k=5)
# X_scaled = selectkbest.fit_transform(X_scaled, y)

# # combine thsese two arrays
# print('Selected features: {}'.format(X.columns[selectkbest.get_support()]))

## 拆分訓練集和測試集

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size = 0.30, random_state = 0)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.30, random_state = 0)

## 開始訓練模型

### 模型：Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, y_train)

# 使用驗證集進行預測
y_pred = lm.predict(X_test)

# 結果可視化
plt.figure(figsize=(10, 5))
plt.plot(y_test.values, marker='o', label='Actual Price')
plt.plot(y_pred, marker='x', label='Predicted Price')
plt.legend()
plt.show()

# test accuracy of the model
print('Accuracy of the model:', lm.score(X_test, y_test))

## 評估模型的好壞

### 回歸模型的評估指標

1. **MSE (Mean Squared Error) - 均方誤差**

   - **解釋**：MSE 是預測值與真實值之間平方誤差的平均值。它反映了模型預測誤差的平方平均值。
   - **意義**：MSE 對於較大的誤差更加敏感，因為誤差被平方了。MSE 越小，表示模型的預測越準確。

2. **MAE (Mean Absolute Error) - 平均絕對誤差**

   - **解釋**：MAE 是預測值與真實值之間絕對誤差的平均值。
   - **意義**：MAE 是一個容易解釋的指標，因為它表示了平均預測誤差的實際值。MAE 越小，表示模型的預測越準確。

3. **RMSE (Root Mean Squared Error) - 均方根誤差**

   - **解釋**：RMSE 是 MSE 的平方根，提供了一個與原始數據單位相同的誤差指標。
   - **意義**：RMSE 與 MSE 類似，但由於它取了平方根，因此與原始數據的尺度一致。RMSE 越小，表示模型的預測越準確。

4. **MAPE (Mean Absolute Percentage Error) - 平均絕對百分比誤差**

   - **解釋**：MAPE 是預測值與真實值之間相對誤差的平均值，通常以百分比表示。
   - **意義**：MAPE 提供了預測誤差的相對尺度，這對於不同量級的數據特別有用。MAPE 越小，表示模型的預測越準確。

#### 綜合理解：
- **MSE** 和 **RMSE** 對於較大的誤差更加敏感，因此它們能夠突出模型在大誤差情況下的性能。
- **MAE** 提供了一個直觀的平均誤差值，易於解釋和理解。
- **MAPE** 提供了預測誤差的相對尺度，特別適合於不同量級的數據比較。

In [ ]:
# from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

print('MSE(Mean Square Error):',mean_absolute_error(y_test,y_pred)) #計算 MAE
print('MAE(Mean Absolute Error):',mean_squared_error(y_test,y_pred)) #計算 MSE
print('RMSE(Root Mean Square Error):',np.sqrt(mean_squared_error(y_test,y_pred))) #計算 RMSE
print('MAPE(Mean Absolute Percentage Error):',mean_absolute_percentage_error(y_test,y_pred)) #計算 MAPE 

### Defining function for regression metrics

#### 1. R² Score (R平方值)
**用途與意義**：
- **用途**：R² Score 是用來衡量模型解釋變數之間變異程度的指標。它表示自變量（輸入變量）能解釋的應變量（輸出變量）總變異的比例。
- **意義**：R² 的值介於 0 和 1 之間。值越接近 1，表示模型越能解釋應變量的變異；值接近 0，表示模型解釋能力很低。具體來說，R² 值為 0.8 表示模型解釋了 80% 的應變量變異。

#### 2. Adjusted R² Score (調整後的 R平方值)
**用途與意義**：
- **用途**：Adjusted R² Score 是對 R² Score 的修正，考慮了模型中變數數量的影響。它用於比較不同複雜度的模型，特別是當模型包含多個自變量時。
- **意義**：Adjusted R² 可以防止過度擬合（overfitting），因為它會隨著不相關變數的加入而減少。它的計算方式調整了變數數量的影響，提供更準確的模型解釋能力評估。

#### 3. Cross Validated R² Score (交叉驗證 R平方值)
**用途與意義**：
- **用途**：Cross Validated R² Score 是通過交叉驗證技術計算的 R² Score，旨在評估模型在未見數據上的表現。
- **意義**：這個指標能夠提供模型在不同數據集上的穩定性評估，減少過度擬合的風險。透過 K-fold 交叉驗證等方法，可以更可靠地估計模型的泛化能力。

#### 4. RMSE (Root Mean Squared Error, 均方根誤差)
**用途與意義**：
- **用途**：RMSE 是用來衡量模型預測值與實際值之間差異的指標。它表示預測誤差的標準差。
- **意義**：RMSE 的值越小，表示模型預測的準確性越高。它對大誤差較為敏感，因為誤差是平方後再取平均值。RMSE 提供了預測誤差的絕對量度，是評估模型預測準確性的重要指標。

In [ ]:
# from sklearn.model_selection import cross_val_score

def Reg_Models_Evaluation_Metrics (model,X_train,y_train,X_test,y_test,y_pred):
    cv_score = cross_val_score(estimator = model, X = X_train, y = y_train, cv = 10)
    
    # Calculating Adjusted R-squared
    r2 = model.score(X_test, y_test)
    # Number of observations is the shape along axis 0
    n = X_test.shape[0]
    # Number of features (predictors, p) is the shape along axis 1
    p = X_test.shape[1]
    # Adjusted R-squared formula
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    R2 = model.score(X_test, y_test)
    CV_R2 = cv_score.mean()

    return R2, adjusted_r2, CV_R2, RMSE
    
    print('RMSE:', round(RMSE,4))
    print('R2:', round(R2,4))
    print('Adjusted R2:', round(adjusted_r2, 4) )
    print("Cross Validated R2: ", round(cv_score.mean(),4) )

In [ ]:
ndf = [Reg_Models_Evaluation_Metrics(lm,X_train,y_train,X_test,y_test,y_pred)]

lm_score = pd.DataFrame(data = ndf, columns=['R2 Score','Adjusted R2 Score','Cross Validated R2 Score','RMSE'])
lm_score.insert(0, 'Model', 'Linear Regression')
lm_score

## 模型：Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Creating and training model
RandomForest_reg = RandomForestRegressor(n_estimators = 10, random_state = 0)

In [ ]:
RandomForest_reg.fit(X_train, y_train)
# Model making a prediction on test data
y_pred = RandomForest_reg.predict(X_test)

In [ ]:
ndf = [Reg_Models_Evaluation_Metrics(RandomForest_reg,X_train,y_train,X_test,y_test,y_pred)]

rf_score = pd.DataFrame(data = ndf, columns=['R2 Score','Adjusted R2 Score','Cross Validated R2 Score','RMSE'])
rf_score.insert(0, 'Model', 'Random Forest')
rf_score

## 模型：Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

# Creating and training model
ridge_reg = Ridge(alpha=3, solver="cholesky")

In [ ]:
ridge_reg.fit(X_train, y_train)
# Model making a prediction on test data
y_pred = ridge_reg.predict(X_test)

In [ ]:
ndf = [Reg_Models_Evaluation_Metrics(ridge_reg,X_train,y_train,X_test,y_test,y_pred)]

rr_score = pd.DataFrame(data = ndf, columns=['R2 Score','Adjusted R2 Score','Cross Validated R2 Score','RMSE'])
rr_score.insert(0, 'Model', 'Ridge Regression')
rr_score

## 模型：XGBoost

In [ ]:
from xgboost import XGBRegressor
# create an xgboost regression model
XGBR = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.8, colsample_bytree=0.8)

In [ ]:
XGBR.fit(X_train, y_train)
# Model making a prediction on test data
y_pred = XGBR.predict(X_test)

In [ ]:
ndf = [Reg_Models_Evaluation_Metrics(XGBR,X_train,y_train,X_test,y_test,y_pred)]

XGBR_score = pd.DataFrame(data = ndf, columns=['R2 Score','Adjusted R2 Score','Cross Validated R2 Score','RMSE'])
XGBR_score.insert(0, 'Model', 'XGBoost')
XGBR_score

## 模型：Recursive Feature Elimination (RFE)

RFE is a wrapper-type feature selection algorithm. This means that a different machine learning algorithm is given and used in the core of the method, is wrapped by RFE, and used to help select features.

Random Forest has usually good performance combining with RFE

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline

# create pipeline
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=60)
model = RandomForestRegressor()
rf_pipeline = Pipeline(steps=[('s',rfe),('m',model)])

In [ ]:
rf_pipeline.fit(X_train, y_train)
# Model making a prediction on test data
y_pred = rf_pipeline.predict(X_test)

In [ ]:
ndf = [Reg_Models_Evaluation_Metrics(rf_pipeline,X_train,y_train,X_test,y_test,y_pred)]

rfe_score = pd.DataFrame(data = ndf, columns=['R2 Score','Adjusted R2 Score','Cross Validated R2 Score','RMSE'])
rfe_score.insert(0, 'Model', 'Random Forest with RFE')
rfe_score

# 結果比較

In [ ]:
predictions = pd.concat([rfe_score, XGBR_score, rr_score, rf_score, lm_score], ignore_index=True, sort=False)
predictions

In [ ]:
f, axe = plt.subplots(1,1, figsize=(18,6))

predictions.sort_values(by=['Cross Validated R2 Score'], ascending=False, inplace=True)

sns.barplot(x='Cross Validated R2 Score', y='Model', data = predictions, ax = axe)
axe.set_xlabel('Cross Validated R2 Score', size=16)
axe.set_ylabel('Model')
axe.set_xlim(0,1.0)

axe.set(title='Model Performance')

plt.show()